In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
!pip install turicreate

    100% |████████████████████████████████| 86.1MB 399kB/s eta 0:00:01
    100% |████████████████████████████████| 327kB 29.7MB/s ta 0:00:01
    100% |████████████████████████████████| 2.7MB 11.3MB/s ta 0:00:01 |█████████████████████████▋      | 2.2MB 57.6MB/s eta 0:00:01
    100% |████████████████████████████████| 23.8MB 1.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for prettytable ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Running setup.py bdist_wheel for resampy ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built prettytable resampy
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

In [4]:
# The code was removed by Watson Studio for sharing.

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode
0,27270,27270,7,01/12/10 8:26,2.55,PX,85123AY
1,27270,27270,7,01/12/10 8:26,3.39,PX,71053R
2,27270,27270,9,01/12/10 8:26,2.75,PX,84406BH
3,27270,27270,7,01/12/10 8:26,3.39,PX,84029GV
4,27270,27270,7,01/12/10 8:26,3.39,PX,84029EX


In [5]:
transactions = df_data_1[['CustomerID','StockCode']]
transactions = transactions.rename(columns={'CustomerID': 'customerId', 'StockCode':'products'})
transactions.head()

,customerId,products
0,27270,85123AY
1,27270,71053R
2,27270,84406BH
3,27270,84029GV
4,27270,84029EX


In [6]:
#Insert TEST data in this cell using "Insert to code" on the right
body = client_83b43f68eb9348f8a7795f9fdefa8efd.get_object(Bucket='recommendationsystemforpurchaseda-donotdelete-pr-l0bytgn7jogfsi',Key='test_J1hm2KQ.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95


In [7]:
customers=df_data_2[['CustomerID']]
customers=customers.drop_duplicates('CustomerID')
customers = customers.rename(columns={'CustomerID': 'customerId'})
customers.head()

,customerId
0,127269
1,227268
6,327267
7,162738
16,252747


In [8]:
print(customers.shape)
customers.head()

(628, 1)


,customerId
0,127269
1,227268
6,327267
7,162738
16,252747


In [9]:
print(transactions.shape)
transactions.head()

(330575, 2)


,customerId,products
0,27270,85123AY
1,27270,71053R
2,27270,84406BH
3,27270,84029GV
4,27270,84029EX


## 3. Data preparation
* Our goal here is to break down each list of items in the `products` column into rows and count the number of products bought by a user

In [10]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [i for i in x.split('|')])
transactions.head(10).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0
0,27270,85123AY
1,27270,71053R
2,27270,84406BH
3,27270,84029GV
4,27270,84029EX
5,27270,22752G
6,27270,21730R
7,427266,21756Q
8,462735,22728B
9,462735,22727B


In [11]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,27270,71053R,1
1,27270,85123AY,1


### 3.1. Create data with user, item, and target field
* This table will be an input for our modeling later
    * In this case, our user is `customerId`, `productId`, and `purchase_count`

In [12]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
# data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 1.1 minutes


In [13]:
print(data.shape)
data.head(20)

(271031, 3)


,customerId,productId,purchase_count
0,0,15056BLH,1
1,0,15056ND,1
2,0,15058CH,1
3,0,15060BB,1
4,0,16218Y,1
5,0,16235A,1
6,0,16236S,1
7,0,17091JD,1
8,0,20674R,1
9,0,20675E,1


### 3.2. Create dummy
* Dummy for marking whether a customer bought that item or not.
* If one buys an item, then `purchase_dummy` are marked as 1
* Why create a dummy instead of normalizing it, you ask?
    * Normalizing the purchase count, say by each user, would not work because customers may have different buying frequency don't have the same taste
    * However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [14]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [15]:
data_dummy = create_data_dummy(data)

### 3.3. Normalize item values across users
* To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [16]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head(20)

productId,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,...,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
(df_matrix.shape)

(972, 3810)

In [18]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head(20)

(972, 3810)


productId,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,...,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head(20)

(256976, 3)


,customerId,productId,scaled_purchase_freq
38,25470,10002F,0.0
99,50949,10002F,0.0
114,56340,10002F,0.0
132,64539,10002F,0.0
140,68229,10002F,0.0
197,104499,10002F,0.0
216,120879,10002F,0.0
264,141759,10002F,0.0
268,142758,10002F,0.0
288,153639,10002F,0.0


#### Define a function for normalizing data

In [20]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

## 4. Split train and test set
* Splitting the data into training and testing sets is an important part of evaluating predictive modeling, in this case a collaborative filtering model. Typically, we use a larger portion of the data for training and a smaller portion for testing. 
* We use 80:20 ratio for our train-test set size.
* Our training portion will be used to develop a predictive model, while the other to evaluate the model's performance.
* Now that we have three datasets with purchase counts, purchase dummy, and scaled purchase counts, we would like to split each.

In [21]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(216824, 3) (54207, 3)


In [22]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [23]:
train_data

customerId,productId,purchase_count
109089,22770L,1
435456,22041A,1
362826,21326V,1
283617,84347V,1
9990,22161W,1
21780,23546H,1
208188,22069H,2
181908,21380E,1
136359,84799U,1
164538,85036AJ,1


In [24]:
test_data

customerId,productId,purchase_count
379116,23157X,1
296397,23381G,1
190908,22846O,1
284517,22955C,1
190908,21948F,1
412776,21494K,1
293607,21452L,1
527265,22061J,3
43560,23444U,1
139959,20832P,1


#### Define a `split_data` function for splitting data to training and test set

In [25]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [26]:
# # lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Baseline Model
Before running a more complicated approach such as collaborative filtering, we would like to use a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity.

### 5.1. Using a Popularity model as a baseline
* The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
* We use `turicreate` library for running and evaluating both baseline and collaborative filtering models below
* Training data is used for model selection

#### Using purchase counts

In [27]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [28]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 216824 observations with 971 users and 3768 items.

Data prepared in: 10.5373s

216824 observations to process; with 3768 unique items.

In [29]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/330575 queries. users per second: 201857

recommendations finished on 2000/330575 queries. users per second: 93694.4

recommendations finished on 3000/330575 queries. users per second: 29897.5

recommendations finished on 4000/330575 queries. users per second: 25203

recommendations finished on 5000/330575 queries. users per second: 22935.1

...

recommendations finished on 327000/330575 queries. users per second: 21600.8

recommendations finished on 328000/330575 queries. users per second: 21633.7

recommendations finished on 329000/330575 queries. users per second: 21619.1

recommendations finished on 330000/330575 queries. users per second: 21613.2

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|   27270    |  81953BF  |        2.0         |  1   |
|   27270    |   20725N  | 1.979539641943734  |  2   |
|   27270    |   22720A  | 1.860349127182045  |  3   |
|   27270    |   21212D  | 1.8356164383561644 |  4   |
|   27270    |   84879M  | 1.8210526315789475 |  5   |
|   27270    |   23442T  |        1.8         |  6   |
|   27270    |   47469B  |        1.8         |  7   |
|   27270    |   22386V  | 1.739884393063584  |  8   |
|   27270    |   22469S  | 1.7241379310344827 |  9   |
|   27270    |   22086O  | 1.6916890080428955 |  10  |
|   27270    |  81953BF  |        2.0         |  1   |
|   27270    |   20725N  | 1.979539641943734  |  2   |
|   27270    |   22720A  | 1.860349127182045  |  3   |
|   27270    |   21212D  | 1.8356164383561644 |  4   |
|   27270    |   84879M  | 1.8210526315789475 |  5   |
|   27270 

#### Define a `model` function for model selection

In [30]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [31]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [32]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 971 users and 3784 items.

Data prepared in: 0.918445s

216824 observations to process; with 3784 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   127269   |   20724F  |  1.0  |  1   |
|   127269   |  84596FA  |  1.0  |  2   |
|   127269   |   21232B  |  1.0  |  3   |
|   127269   |   23263V  |  1.0  |  4   |
|   127269   |   22144E  |  1.0  |  5   |
|   127269   |   22992G  |  1.0  |  6   |
|   127269   |   23497Q  |  1.0  |  7   |
|   127269   |   21809L  |  1.0  |  8   |
|   127269   |   22342F  |  1.0  |  9   |
|   127269   |   22960K  |  1.0  |  10  |
|   227268   |   20724F  |  1.0  |  1   |
|   227268   |  84596FA  |  1.0  |  2   |
|   227268   |   21232B  |  1.0  |  3   |
|   227268   |   23263V  |  1.0  |  4   |
|   227268   |   22144E  |  1.0  |  5   |
|   227268   |   22992G  |  1.0  |  6   |
|   227268   |   23497Q  |  1.0  |  7   |
|   227268   |   21809L  |  1.0  |  8   |
|   227268   |   22342F  |  1.0  |  9   |
|   227268   |   22960K  |  1.0  |  10  |
|   327267   |   20724F  |  1.0  |

#### Using normalized purchase count

In [33]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 205580 observations with 969 users and 2494 items.

Data prepared in: 0.881941s

205580 observations to process; with 2494 unique items.

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|   127269   |   84548X  |         1.0         |  1   |
|   127269   |  84751BL  |         0.5         |  2   |
|   127269   |   21816Y  |         0.5         |  3   |
|   127269   |  81950BS  |         0.4         |  4   |
|   127269   |  72369AK  |  0.3333333333333333 |  5   |
|   127269   |   85189T  |  0.3333333333333333 |  6   |
|   127269   |   21393V  |  0.3333333333333333 |  7   |
|   127269   |   21196B  |  0.3333333333333333 |  8   |
|   127269   |   21841K  |  0.2857142857142857 |  9   |
|   127269   |   23442T  | 0.26666666666666666 |  10  |
|   227268   |   84548X  |         1.0         |  1   |
|   227268   |  84751BL  |         0.5         |  2   |
|   227268   |   21816Y  |         0.5         |  3   |
|   227268   |  81950BS  |         0.4         |  4   |
|   227268   |  72369AK  |  0.3333333333333333 |

#### Notes
* Once we created the model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks. 
* In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.
* If a grouping example below, products 132, 248, 37, and 34 are the most popular (best-selling) across customers. Using their purchase counts divided by the number of customers, we see that these products are at least bought 3 times on average in the training set of transactions (same as the first popularity measure on `purchase_count` variable)

In [34]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
22423U     2.384783
85123AY    2.338266
85099BJ    2.309829
47566Y     2.021635
81953BF    2.000000
20725N     1.979540
22720A     1.860349
21212D     1.835616
22383K     1.835227
84879M     1.821053
23298Q     1.810496
22197J     1.800983
23442T     1.800000
47469B     1.800000
20727V     1.783784
22386V     1.739884
20728Z     1.729282
22469S     1.724138
22699D     1.723810
22382I     1.717579
Name: purchase_count, dtype: float64

## 6. Collaborative Filtering Model

* In collaborative filtering, we would recommend items based on how similar users purchase items. For instance, if customer 1 and customer 2 bought similar items, e.g. 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.

* To define similarity across users, we use the following steps:
    1. Create a user-item matrix, where index values represent unique customer IDs and column values represent unique product IDs
    
    2. Create an item-to-item similarity matrix. The idea is to calculate how similar a product is to another product. There are a number of ways of calculating this. In steps 6.1 and 6.2, we use cosine and pearson similarity measure, respectively.  
    
        * To calculate similarity between products X and Y, look at all customers who have rated both these items. For example, both X and Y have been rated by customers 1 and 2. 
        * We then create two item-vectors, v1 for item X and v2 for item Y, in the user-space of (1, 2) and then find the `cosine` or `pearson` angle/distance between these vectors. A zero angle or overlapping vectors with cosine value of 1 means total similarity (or per user, across all items, there is same rating) and an angle of 90 degree would mean cosine of 0 or no similarity.
        
    3. For each customer, we then predict his likelihood to buy a product (or his purchase counts) for products that he had not bought. 
    
        * For our example, we will calculate rating for user 2 in the case of item Z (target item). To calculate this we weigh the just-calculated similarity-measure between the target item and other items that customer has already bought. The weighing factor is the purchase counts given by the user to items already bought by him. 
        * We then scale this weighted sum with the sum of similarity-measures so that the calculated rating remains within a predefined limits. Thus, the predicted rating for item Z for user 2 would be calculated using similarity measures.

* While I wrote python scripts for all the process including finding similarity using python scripts (which can be found in `scripts` folder, we can use `turicreate` library for now to capture different measures like using `cosine` and `pearson` distance, and evaluate the best model.

### 6.1. `Cosine` similarity
* Similarity is the cosine of the angle between the 2 vectors of the item vectors of A and B
* It is defined by the following formula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Closer the vectors, smaller will be the angle and larger the cosine

#### Using purchase count

In [35]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216824 observations with 971 users and 3768 items.

Data prepared in: 0.943009s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.69042s

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|   127269   |  85099BJ  |  0.2690374517440796 |  1   |
|   127269   |  85123AY  | 0.26133861422538757 |  2   |
|   127269   |   22383K  |  0.2590727913379669 |  3   |
|   127269   |   22697A  |    0.258310546875   |  4   |
|   127269   |   22423U  | 0.25731443405151366 |  5   |
|   127269   |   22382I  |  0.250646071434021  |  6   |
|   127269   |   47566Y  | 0.25055856466293336 |  7   |
|   127269   |   22469S  | 0.24641125202178954 |  8   |
|   127269   |   23203B  | 0.24285009860992432 |  9   |
|   127269   |   20725N  | 0.24239929914474487 |  10  |
|   227268   |  85099BJ  |  0.2690374517440796 |  1   |
|   227268   |  85123AY  | 0.26133861422538757 |  2   |
|   227268   |   22383K  |  0.2590727913379669 |  3   |
|   227268   |   22697A  |    0.258310546875   |  4   |
|   227268   |   22423U  | 0.25731443405151366 |

#### Using purchase dummy

In [36]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 971 users and 3784 items.

Data prepared in: 0.9216s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.04833s

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   127269   |   20724F  |  0.0  |  1   |
|   127269   |  84596FA  |  0.0  |  2   |
|   127269   |   21232B  |  0.0  |  3   |
|   127269   |   23263V  |  0.0  |  4   |
|   127269   |   22144E  |  0.0  |  5   |
|   127269   |   22992G  |  0.0  |  6   |
|   127269   |   23497Q  |  0.0  |  7   |
|   127269   |   21809L  |  0.0  |  8   |
|   127269   |   22342F  |  0.0  |  9   |
|   127269   |   22960K  |  0.0  |  10  |
|   227268   |   20724F  |  0.0  |  1   |
|   227268   |  84596FA  |  0.0  |  2   |
|   227268   |   21232B  |  0.0  |  3   |
|   227268   |   23263V  |  0.0  |  4   |
|   227268   |   22144E  |  0.0  |  5   |
|   227268   |   22992G  |  0.0  |  6   |
|   227268   |   23497Q  |  0.0  |  7   |
|   227268   |   21809L  |  0.0  |  8   |
|   227268   |   22342F  |  0.0  |  9   |
|   227268   |   22960K  |  0.0  |  10  |
|   327267   |   20724F  |  0.0  |

#### Using normalized purchase count

In [37]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 205580 observations with 969 users and 2494 items.

Data prepared in: 0.944034s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.88963s

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|   127269   |  85123AY  | 0.05021348595619202  |  1   |
|   127269   |  85099BJ  | 0.04971571207046509  |  2   |
|   127269   |   22423U  | 0.048471102714538576 |  3   |
|   127269   |   22197J  | 0.04611009120941162  |  4   |
|   127269   |   20725N  | 0.04481416583061218  |  5   |
|   127269   |   22411U  | 0.04312252521514893  |  6   |
|   127269   |   22384W  | 0.041453471183776854 |  7   |
|   127269   |   22960K  | 0.041333638429641724 |  8   |
|   127269   |   23203B  | 0.04114018201828003  |  9   |
|   127269   |   22383K  | 0.04099071741104126  |  10  |
|   227268   |  85123AY  | 0.05021348595619202  |  1   |
|   227268   |  85099BJ  | 0.04971571207046509  |  2   |
|   227268   |   22423U  | 0.048471102714538576 |  3   |
|   227268   |   22197J  | 0.04611009120941162  |  4   |
|   227268   |   20725N  | 0.04

### 6.2. `Pearson` similarity
* Similarity is the pearson coefficient between the two vectors.
* It is defined by the following formula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Using purchase count

In [38]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216824 observations with 971 users and 3768 items.

Data prepared in: 0.681085s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 46.7293s

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|   127269   |   22423U  | 2.2210108518082157 |  1   |
|   127269   |  85099BJ  | 2.1693495368855635 |  2   |
|   127269   |  85123AY  | 2.1596159900998995 |  3   |
|   127269   |  81953BF  |        2.0         |  4   |
|   127269   |   47566Y  | 1.9410298046698955 |  5   |
|   127269   |   20725N  | 1.8266178949745124 |  6   |
|   127269   |   22720A  | 1.8054334570255663 |  7   |
|   127269   |   23442T  |        1.8         |  8   |
|   127269   |   47469B  |        1.8         |  9   |
|   127269   |   21212D  | 1.7760234594018494 |  10  |
|   227268   |   22423U  | 2.2210108518082157 |  1   |
|   227268   |  85099BJ  | 2.1693495368855635 |  2   |
|   227268   |  85123AY  | 2.1596159900998995 |  3   |
|   227268   |  81953BF  |        2.0         |  4   |
|   227268   |   47566Y  | 1.9410298046698955 |  5   |
|   227268

#### Using purchase dummy

In [39]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 971 users and 3784 items.

Data prepared in: 0.858861s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 46.5333s

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   127269   |   20724F  |  0.0  |  1   |
|   127269   |  84596FA  |  0.0  |  2   |
|   127269   |   21232B  |  0.0  |  3   |
|   127269   |   23263V  |  0.0  |  4   |
|   127269   |   22144E  |  0.0  |  5   |
|   127269   |   22992G  |  0.0  |  6   |
|   127269   |   23497Q  |  0.0  |  7   |
|   127269   |   21809L  |  0.0  |  8   |
|   127269   |   22342F  |  0.0  |  9   |
|   127269   |   22960K  |  0.0  |  10  |
|   227268   |   20724F  |  0.0  |  1   |
|   227268   |  84596FA  |  0.0  |  2   |
|   227268   |   21232B  |  0.0  |  3   |
|   227268   |   23263V  |  0.0  |  4   |
|   227268   |   22144E  |  0.0  |  5   |
|   227268   |   22992G  |  0.0  |  6   |
|   227268   |   23497Q  |  0.0  |  7   |
|   227268   |   21809L  |  0.0  |  8   |
|   227268   |   22342F  |  0.0  |  9   |
|   227268   |   22960K  |  0.0  |  10  |
|   327267   |   20724F  |  0.0  |

#### Using normalized purchase count

In [40]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 205580 observations with 969 users and 2494 items.

Data prepared in: 0.960326s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 40.8085s

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|   127269   |   84548X  |         1.0         |  1   |
|   127269   |   21816Y  |         0.5         |  2   |
|   127269   |  84751BL  |         0.5         |  3   |
|   127269   |  81950BS  |         0.4         |  4   |
|   127269   |  72369AK  |  0.3333333333333333 |  5   |
|   127269   |   21393V  |  0.3333333333333333 |  6   |
|   127269   |   85189T  |  0.3333333333333333 |  7   |
|   127269   |   21196B  |  0.3333333333333333 |  8   |
|   127269   |   21841K  | 0.28571428571428575 |  9   |
|   127269   |   23442T  | 0.26666666666666666 |  10  |
|   227268   |   84548X  |         1.0         |  1   |
|   227268   |   21816Y  |         0.5         |  2   |
|   227268   |  84751BL  |         0.5         |  3   |
|   227268   |  81950BS  |         0.4         |  4   |
|   227268   |  72369AK  |  0.3333333333333333 |

#### Note
* In collaborative filtering above, we used two approaches: cosine and pearson distance. We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.
* We can see that the recommendations are different for each user. This suggests that personalization does exist. 
* But how good is this model compared to the baseline, and to each other? We need some means of evaluating a recommendation engine. Lets focus on that in the next section.

## 7. Model Evaluation
For evaluating recommendation engines, we can use the concept of precision-recall.

* RMSE (Root Mean Squared Errors)
    * Measures the error of predicted values
    * Lesser the RMSE value, better the recommendations
* Recall
    * What percentage of products that a user buys are actually recommended?
    * If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6
* Precision
    * Out of all the recommended items, how many the user actually liked?
    * If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8
    
* Why are both recall and precision important?
    * Consider a case where we recommend all products, so our customers will surely cover the items that they liked and bought. In this case, we have 100% recall! Does this mean our model is good?
    * We have to consider precision. If we recommend 300 items but user likes and buys only 3 of them, then precision is 0.1%! This very low precision indicates that the model is not great, despite their excellent recall.
    * So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

Lets compare all the models we have built based on precision-recall characteristics:

In [41]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Models on purchase counts

In [42]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts

Precision and recall summary statistics by cutoff
+--------+----------------------+---------------------+
| cutoff |     mean_recall      |    mean_precision   |
+--------+----------------------+---------------------+
|   1    | 0.004929836558357202 | 0.27619047619047654 |
|   2    | 0.009460932663440703 | 0.24232804232804228 |
|   3    | 0.01294623755877099  | 0.22751322751322753 |
|   4    | 0.015580788032447904 |  0.2145502645502644 |
|   5    | 0.018175052876284053 | 0.19915343915343917 |
|   6    | 0.022347337295660186 | 0.19135802469135788 |
|   7    | 0.02491299539658061  | 0.18760393046107335 |
|   8    | 0.02781621125704707  |  0.1850529100529101 |
|   9    | 0.03143302165109048  | 0.18683127572016447 |
|   10   | 0.03459666840426081  |  0.1879365079365075 |
+--------+----------------------+---------------------+
[10 rows x 3 columns]



Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 56)"

Note that increasing the degree of parallelism also increases the memory footprint.


Overall RMSE: 0.5284507042470965

Per User RMSE (best)
+------------+----------------------+-------+
| customerId |         rmse         | count |
+------------+----------------------+-------+
|   471816   | 0.033759108960261096 |   4   |
+------------+----------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+------------+--------------------+-------+
| customerId |        rmse        | count |
+------------+--------------------+-------+
|   419976   | 1.2497816081623085 |  124  |
+------------+--------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+-----------+------+-------+
| productId | rmse | count |
+-----------+------+-------+
|   51008P  | 0.0  |   6   |
+-----------+------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+-----------+--------------------+-------+
| productId |        rmse        | count |
+-----------+--------------------+-------+
|  85123AY  | 1.6273447039728817 |  126  |
+-----------+--------------------+-------+


#### Models on purchase dummy

In [43]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy

Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |      mean_recall      |    mean_precision    |
+--------+-----------------------+----------------------+
|   1    | 0.0001851043331336039 | 0.010626992561105199 |
|   2    | 0.0002684889035515802 | 0.008501594048884172 |
|   3    | 0.0005203156938603141 | 0.008147360963513994 |
|   4    | 0.0006170253393866926 | 0.008501594048884167 |
|   5    | 0.0007261072046298127 | 0.008289054197662054 |
|   6    | 0.0007680181904178396 | 0.007616011335458735 |
|   7    | 0.0009403951264168706 |  0.0080461515105511  |
|   8    | 0.0010129827084559252 | 0.007970244420828899 |
|   9    | 0.0012354418027094294 | 0.008029283268390594 |
|   10   | 0.0013252341857038254 | 0.008182784272051007 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+------------

#### Models on normalized purchase frequency

In [44]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+------------------------+-----------------------+
| cutoff |      mean_recall       |     mean_precision    |
+--------+------------------------+-----------------------+
|   1    | 7.707782548038738e-06  | 0.0010559662090813117 |
|   2    | 3.4324559219555624e-05 |  0.001583949313621966 |
|   3    |  5.94531668688168e-05  | 0.0017599436818021832 |
|   4    |  5.94531668688168e-05  | 0.0013199577613516363 |
|   5    |  5.94531668688168e-05  | 0.0010559662090813104 |
|   6    |  5.94531668688168e-05  | 0.0008799718409010916 |
|   7    | 6.757598386174979e-05  | 0.0009051138934982643 |
|   8    | 7.438866908162923e-05  | 0.0009239704329461454 |
|   9    | 0.00016809780139564386 | 0.0014079549454417482 |
|   10   | 0.00016809780139564386 | 0.0012671594508975703 |
+--------+------------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.2

## 8. Model Selection
### 8.1. Evaluation summary
* Based on RMSE


    1. Popularity on purchase counts: 1.1111750034210488
    2. Cosine similarity on purchase counts: 1.9230643981653215
    3. Pearson similarity on purchase counts: 1.9231102838192284
    
    4. Popularity on purchase dummy: 0.9697374361161925
    5. Cosine similarity on purchase dummy: 0.9697509978436404
    6. Pearson similarity on purchase dummy: 0.9697745320187097
    
    7. Popularity on scaled purchase counts: 0.16230660626840343
    8. Cosine similarity on scaled purchase counts: 0.16229800354111104
    9. Pearson similarity on scaled purchase counts: 0.1622982668334026
    
* Based on Precision and Recall
![](../images/model_comparisons.png)


#### Notes

* Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.
* Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts. However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
* RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine.
Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

## 8. Final Output
* In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.
* We need to first rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [45]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_count', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 271031 observations with 972 users and 3810 items.

Data prepared in: 0.791342s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.8958s

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|   127269   |   20725N  | 0.33143950343132017 |  1   |
|   127269   |  85099BJ  |  0.3276068377494812 |  2   |
|   127269   |   20727V  | 0.31362108945846556 |  3   |
|   127269   |   22697A  | 0.31260724425315856 |  4   |
|   127269   |   22411U  | 0.30559892416000367 |  5   |
|   127269   |   22383K  | 0.30556941509246827 |  6   |
|   127269   |  85123AY  | 0.30389530301094053 |  7   |
|   127269   |   22423U  |  0.3004434359073639 |  8   |
|   127269   |   21212D  | 0.29774430751800535 |  9   |
|   127269   |   22197J  |  0.2937191450595856 |  10  |
|   227268   |   20725N  | 0.33143950343132017 |  1   |
|   227268   |  85099BJ  |  0.3276068377494812 |  2   |
|   227268   |   20727V  | 0.31362108945846556 |  3   |
|   227268   |   22697A  | 0.31260724425315856 |  4   |
|   227268   |   22411U  | 0.30559892416000367 |

### 8.1. CSV output file

In [46]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head(10)

(6280, 4)


,customerId,productId,score,rank
0,127269,20725N,0.331440,1
1,127269,85099BJ,0.327607,2
2,127269,20727V,0.313621,3
3,127269,22697A,0.312607,4
4,127269,22411U,0.305599,5
5,127269,22383K,0.305569,6
6,127269,85123AY,0.303895,7
7,127269,22423U,0.300443,8
8,127269,21212D,0.297744,9
9,127269,22197J,0.293719,10


In [47]:
# df_rec1 = df_rec.to_dataframe()
import json
df2_unique = df_rec.drop_duplicates(subset='customerId')
final_recom = df2_unique.filter(['customerId',str('productId')])
final_recom = final_recom.rename(columns={'customerId': 'CustomerID', 'productId':'Items'})
final_recom['Items'] = final_recom['Items'].apply(lambda x: "'" + str(x) + "'")
final_recom['Items'] = '[' + final_recom['Items'].astype(str) + ']'

In [48]:
project.save_data(data=final_recom.to_csv(index=False,sep=','),file_name='recommended_purchase.csv',overwrite=True)

{'asset_id': 'c55f8ed8-9251-43fc-ab00-638411059644',
 'bucket_name': 'recommendationsystemforpurchaseda-donotdelete-pr-l0bytgn7jogfsi',
 'file_name': 'recommended_purchase.csv',
 'message': 'File saved to project storage.'}

In [49]:
print(final_recom.shape)
final_recom.head(20)

(628, 2)


,CustomerID,Items
0,127269,['20725N']
10,227268,['20725N']
20,327267,['20725N']
30,162738,['20725N']
40,252747,['20725N']
50,347247,['20725N']
60,442755,['20725N']
70,342756,['20725N']
80,57240,['20725N']
90,357237,['20725N']


In [50]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define a function to create a desired output

In [51]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('recommendation_output.csv')
    return df_output

In [52]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
rec_output=df_output.drop_duplicates()
project.save_data(data=rec_output.to_csv(index=False),file_name='recommendation_output.csv',overwrite=True)
print(df_output.shape)
df_output.head(12)

(628, 1)


,recommendedProducts
customerId,
1890,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
2700,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
3600,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
3690,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
4500,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
5490,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
8190,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
9090,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
13680,84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...


### 8.2. Customer recommendation function

In [53]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [54]:
customer_recomendation(19980)

recommendedProducts    84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
Name: 19980, dtype: object

In [55]:
customer_recomendation(15390)

recommendedProducts    84548X|21816Y|84751BL|81950BS|72369AK|21393V|8...
Name: 15390, dtype: object

## Summary
In this exercise, we were able to traverse a step-by-step process for making recommendations to customers. We used Collaborative Filtering approaches with `cosine` and `pearson` measure and compare the models with our baseline popularity model. We also prepared three sets of data that include regular buying count, buying dummy, as well as normalized purchase frequency as our target variable. Using RMSE, precision and recall, we evaluated our models and observed the impact of personalization. Finally, we selected the Cosine approach in dummy purchase data. 